#### Objective : To try hand at feature engineering 
- learn the importance 
- some techniques


Dataset : https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data 

In [18]:
import pandas as pd
df_train = pd.read_csv(r'./Data/feature_engineering/train.csv',index_col='id')

In [19]:
print(df_train.info())
print(df_train.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 7 columns):
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(1)
memory usage: 9.7+ MB
None
               toxic   severe_toxic        obscene         threat  \
count  159571.000000  159571.000000  159571.000000  159571.000000   
mean        0.095844       0.009996       0.052948       0.002996   
std         0.294379       0.099477       0.223931       0.054650   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         0.000000       0.000000       0.000000       0

In [3]:
# checking the null values
print(df_train.isnull().sum())

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


In [20]:
df_train.tail()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [21]:
df_train['comment_text'].tail()

id
ffe987279560d7ff    ":::::And for the second time of asking, when ...
ffea4adeee384e90    You should be ashamed of yourself \n\nThat is ...
ffee36eab5c267c9    Spitzer \n\nUmm, theres no actual article for ...
fff125370e4aaaf3    And it looks like it was actually you who put ...
fff46fc426af1f9a    "\nAnd ... I really don't think you understand...
Name: comment_text, dtype: object

In [14]:
df_train.loc[:,['toxic','severe_toxic','obscene','threat','insult','identity_hate']].apply(sum, axis=0)

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [23]:
#CLEANING UP THE TEXT
#Function to clean up the text
def standardizetext(df, textfield):
    df[textfield] = df[textfield].str.replace(r"http\S+", "")
    df[textfield] = df[textfield].str.replace(r"http", "")
    df[textfield] = df[textfield].str.replace(r"@\S+", "")
    df[textfield] = df[textfield].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[textfield] = df[textfield].str.replace(r"@", "at")
    df[textfield] = df[textfield].str.lower()
    return df

In [24]:
df_train = standardizetext(df_train, "comment_text")
df_train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,explanation\nwhy the edits made under my usern...,0,0,0,0,0,0
000103f0d9cfb60f,d'aww! he matches this background colour i'm s...,0,0,0,0,0,0
000113f07ec002fd,"hey man, i'm really not trying to edit war it...",0,0,0,0,0,0
0001b41b1c6bb37e,"""\nmore\ni can't make any real suggestions on ...",0,0,0,0,0,0
0001d958c54c6e35,"you, sir, are my hero any chance you remember...",0,0,0,0,0,0


### Summary -- Take Aways


#### References :
https://www.kaggle.com/eikedehling/feature-engineering
